# Caso de Estadística Descriptiva usando Python

Abr-2021

*Por: Andrés Felipe Escallón Portilla*



**Caso**

En una graja agrícola que posee un sistema de ventilación natural, se han tomado varias mediciones de la concentración de Amoniaco (NH3) así como también, datos sobre la Velocidad del viento, Temperatura y Humedad relativa durante el día 5 de marzo de 2014. Los datos se encuentran disponibles en: [amoniaco](https://drive.google.com/file/d/1n0maNQ5Kzve4iF-v-hbbGHPkzoy_0m7k/view?usp=sharing)

1. Realice un informe donde incluya un análisis descriptivo de las muestras de las variables Velocidad del Viento y Concentraciones de Amoniaco por separado, conservando la coherencia según la naturaleza de las variables, eliminando datos atípicos (si los hay) e interpretando las principales medidas de resumen (indicadores).

2. Explique mediante un modelo de regresión lineal simple, el comportamiento conjunto de ambas variables. Detalle el modelo y construya la recta de regresión dentro del diagrama de dispersión.

# Solución:

In [ ]:
# Para poder cargar archivos desde Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Instalando los paquetes requeridos
!pip install -r '/content/drive/MyDrive/Taller-Estadistica-Descriptiva-Python/requirements.txt'

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/drive/MyDrive/Taller-Estadistica-Descriptiva-Python/requirements.txt'


In [ ]:
#English version:
# Importing all the required packages

import pandas              as pd # The gold standard of Python data analysis, to create and manipulate tables of data
import numpy               as np # The Python module for processing arrays which/Pandas is based on
import seaborn             as sns; sns.set() # A package to make Matplotlib visualizations more aesthetic
import branca
import geopandas

import matplotlib.pyplot   as plt # The gold standard of Python data visualization, but can be complex to use
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.patches import Patch
from matplotlib.widgets import Slider, Button, RadioButtons

import statsmodels.api as sm
import statsmodels.formula.api as sfm
from statsmodels.formula.api import ols

import scipy
from scipy import stats
from scipy import interp
from scipy.optimize import fsolve
from scipy.stats import chi2_contingency, ttest_ind, norm # A module for Python machine learning--we'll stick to T-Tests here

import sklearn
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, RobustScaler, StandardScaler
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from statsmodels.formula.api import ols

from IPython.display import display
from IPython.display import display_html
from IPython.display import Image, SVG

import folium # package for making maps, please make sure to use a version older than 1.0.0.
from folium.plugins import TimeSliderChoropleth
# from time_slider_choropleth import TimeSliderChoropleth

import json
import requests
from bs4 import BeautifulSoup
import os
import pydotplus
from io import StringIO
from sympy import var, plot_implicit, Eq
from graphviz import Source
from wordcloud import WordCloud # A package that will allow us to make a wordcloud

# when executing, the plot will be done
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (8,5)
import warnings
warnings.filterwarnings('ignore')
# ignore log(0) and divide by 0 warning
np.seterr(divide='ignore');

In [ ]:
0.1+0.1 #Recordar que python trabaja el "." como decimal

0.2

In [ ]:
#leyendo el archivos de la base de datos (guardado previamente como csv) y asignándolo a dataframe:
df_csv=pd.read_csv('/content/drive/MyDrive/Taller-Estadistica-Descriptiva-Python/amoniaco_csv.csv')
df_csv

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Taller-Estadistica-Descriptiva-Python/amoniaco_csv.csv'

https://jdvelasq.github.io/courses/notebooks/pandas/1-05-conversion-de-formatos.html

In [ ]:
#Leyendo directamente desde el archivo excel, pone el formato decimal que Python maneja (.)
df_orig=pd.read_excel('/content/drive/MyDrive/Taller-Estadistica-Descriptiva-Python/amoniaco.xlsx')
df_orig

In [ ]:
#Esta es la base de datos como tal con la cual se va a trabajar
pd.options.display.max_rows = None # para visualizar todas las filas (registros) de interés
df=df_orig.copy()
df

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

In [ ]:
amoniaco_asc = df.sort_values('amoniaco (ppm)')
amoniaco_asc['amoniaco (ppm)']

In [ ]:
amoniaco_asc['amoniaco (ppm)'].sum()

In [ ]:
viento_asc = df.sort_values('viento (m/s )')
viento_asc['viento (m/s )']

In [ ]:
viento_asc['viento (m/s )'].sum()

In [ ]:
df.describe()

In [ ]:
df.describe().shape

In [ ]:
df.describe().columns

In [ ]:
df.describe().index

In [ ]:
df_describe_reset=df.describe().reset_index()
df_describe_reset

In [ ]:
df_describe_reset_transpuesto=df_describe_reset.transpose()
df_describe_reset_transpuesto

In [ ]:
df_describe_transpuesto=df.describe().transpose()
df_describe_transpuesto

In [ ]:
df_describe_transpuesto_final=df_describe_transpuesto.copy()
df_describe_transpuesto_final['range']=df_describe_transpuesto_final['max']-df_describe_transpuesto_final['min']
df_describe_transpuesto_final['variance']=df_describe_transpuesto_final['std']*df_describe_transpuesto_final['std'] #variance(var)=standar_deviation(std)^2
df_describe_transpuesto_final['variation_coef']=df_describe_transpuesto_final['std']/df_describe_transpuesto_final['mean'] #var_coef=std/mean
df_describe_transpuesto_final

In [ ]:
df.head(3)

# Comprobando resultados de la tabla que resume las estadísticas (df.describe()):

In [ ]:
n=len(df)
n

In [ ]:
vars=['humedad (%)', 'temperatura (C°)', 'amoniaco (ppm)', 'viento (m/s )']

In [ ]:
sumas=[df[col].sum() for col in vars] # via rapida
sumas

In [ ]:
promedios=[df[col].mean() for col in vars] # via rapida
promedios

In [ ]:
desviaciones_estandar=[df[col].std() for col in vars] # via rapida
desviaciones_estandar

In [ ]:
promedio1=df['humedad (%)'].sum()/n
promedio1

In [ ]:
df.columns

In [ ]:
#vars=['humedad (%)', 'temperatura (C°)', 'amoniaco (ppm)', 'viento (m/s )']

In [ ]:
suma1=[]
suma1.append(df[vars[0]][0])

for i in range(1,n):# i=index
  suma1.append(df[vars[0]][i]+suma1[i-1])

print('suma1= ',suma1[-1])# el último elemento tiene la suma total (acumulada de todos los elementos)
promedio1=suma1[-1]/n
print('promedio1= ',promedio1)

In [ ]:
suma1=[]
suma2=[]
suma3=[]
suma4=[]

sumas=[suma1, suma2, suma3, suma4]

for j in range(len(vars)):#lista de listas (j=columna)

  sumas[j].append(df[vars[j]][0])

  for i in range(1,n): # i=index (fila)
    sumas[j].append(df[vars[j]][i]+sumas[j][i-1])

print(sumas) #lista de listas
print([sumas[j][-1] for j in range(len(vars))])#imprime las cuatro sumas totales acumuladas


In [ ]:
promedios=[sumas[j][-1]/n for j in range(len(vars))] # j=col
promedios

In [ ]:
promedios=[df[col].sum()/n for col in vars] # via mas rapida
promedios

In [ ]:
df.columns

In [ ]:
len(df.columns)

In [ ]:
#MAX
maximos=[df[j].max() for j in vars] # via mas rapida
maximos

In [ ]:
#MIN
minimos=[df[j].min() for j in vars] # via mas rapida
minimos

In [ ]:
rangos=np.array(maximos)-np.array(minimos) # no se puede restar dos listas directamente (hay que pasarlas a arreglos)
rangos

In [ ]:
#RANGO= MAX-MIN
rangos=[df[j].max()-df[j].min() for j in vars] # via mas rapida
rangos

In [ ]:
df_final=df.copy()
df_final.head(3)

In [ ]:
xi_menos_xprom=[[df[col][fil]-df[col].mean() for fil in range(len(df))] for col in vars] # lista de listas
print(xi_menos_xprom)
print([np.array(xi_menos_xprom[col]).sum() for col in range(len(vars))])# debe ser cero para cada resultado (o muy cercano a cero en el limite por calculo computacional)

In [ ]:
xi_menos_xprom_cuadrado=[np.array(xi_menos_xprom[col])**2 for col in range(len(vars))] #x**2=x^2
print(xi_menos_xprom_cuadrado)
varianzas=[np.array(xi_menos_xprom_cuadrado[col]).sum()/n for col in range(len(vars))]
print(varianzas)# # arroja el resultado de las varianzas de cada columna

In [ ]:
desviaciones_estandar=np.array(varianzas)**0.5 # std=raiz_cuadrada(var)
print(desviaciones_estandar)# # arroja el resultado de las desviaciones_estandar de cada columna

In [ ]:
xi_xprom=['xi-xprom1','xi-xprom2','xi-xprom3','xi-xprom4']
xi_xprom_cuadrado=['(xi-xprom1)^2','(xi-xprom2)^2','(xi-xprom3)^2','(xi-xprom4)^2']

for i,j in enumerate(xi_xprom):
  df_final[j]=xi_menos_xprom[i]

for i,j in enumerate(xi_xprom_cuadrado):
  df_final[j]=xi_menos_xprom_cuadrado[i]

'''
df_final['xi-xprom1']=xi_menos_xprom[0]
df_final['xi-xprom2']=xi_menos_xprom[1]
df_final['xi-xprom3']=
df_final['variance']=df_describe_transpuesto_final['std']*df_describe_transpuesto_final['std'] #variance(var)=standar_deviation(std)^2
df_final['variation_coef']=df_describe_transpuesto_final['std']/df_describe_transpuesto_final['mean'] #var_coef=std/mean
'''
df_final.head(3)


In [ ]:
maximos=df_final[['humedad (%)','temperatura (C°)','amoniaco (ppm)','viento (m/s )']].max()
maximos

In [ ]:
minimos=df_final[['humedad (%)','temperatura (C°)','amoniaco (ppm)','viento (m/s )']].min()
minimos

In [ ]:
rangos=maximos-minimos
rangos

In [ ]:
print([(df_final[j].sum()/n) for j in vars]) #  promedios de las var 1-4 (1: humedad, 2: temperatrura, 3: amoniaco, 4:viento)

In [ ]:
promedios=df_final[['humedad (%)','temperatura (C°)','amoniaco (ppm)','viento (m/s )']].mean()
promedios

In [ ]:
print([df_final[col].sum() for col in xi_xprom]) # debe ser 0 (o muy cercano en el limite)

In [ ]:
print([df_final[col].sum()/n for col in xi_xprom_cuadrado]) #  varianzas

In [ ]:
varianzas=df_final[['humedad (%)','temperatura (C°)','amoniaco (ppm)','viento (m/s )']].var()
varianzas

In [ ]:
print([(df_final[col].sum()/n)**0.5 for col in xi_xprom_cuadrado]).std() #  desviaciones estandar

In [ ]:
desviaciones_estandar=df_final[['humedad (%)','temperatura (C°)','amoniaco (ppm)','viento (m/s )']].std()
desviaciones_estandar

In [ ]:
print(np.array([(df_final[col].sum()/n)**0.5 for col in xi_xprom_cuadrado])/np.array([(df_final[j].sum()/n) for j in vars])) #  coeficientes de variación = desviaciones estandar / promedios

In [ ]:
coef_variacion=desviaciones_estandar/promedios
coef_variacion

In [ ]:
#EJEMPLO PERCENTIL(25, 50 Y 75):
#percentile(25) #percentil 25 (Q1) de la var viento
df.quantile([.1, .25, .5, .75], axis = 0)

In [ ]:
df.quantile(0.89, axis = 0)

# Gráficas: Variable (y) vs Tiempo (x), y=f(x)

In [ ]:
plt.figure(figsize=(25, 8))
ax = sns.lineplot(
    x=df.columns[0],
    y=df.columns[1],
    data=df,
)

In [ ]:
plt.figure(figsize=(25, 8))
ax = sns.lineplot(
    x=df.columns[0],
    y=df.columns[2],
    data=df,
)

In [ ]:
plt.figure(figsize=(25, 8))
ax = sns.lineplot(
    x=df.columns[0],
    y=df.columns[3],
    data=df,
)

In [ ]:
plt.figure(figsize=(25, 8))
ax = sns.lineplot(
    x=df.columns[0],
    y=df.columns[4],
    data=df,
)

# Distribución de las variables: Conteo(y) vs variable(x):

- 'humedad (%)'
- 'temperatura (C°)'
- 'amoniaco (ppm)'
- 'viento (m/s )'

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(50, 8))
sns.countplot(x=df.columns[1], data = df)
plt.title('humedad (%)')

In [ ]:
tabla_frec_humedad=df.sort_values('humedad (%)')['humedad (%)'].value_counts().reset_index().sort_values('index')
tabla_frec_humedad

In [ ]:
tabla_frec_humedad.sum()# n=63 ; sum_x must be corrected as in the next line of code

In [ ]:
(tabla_frec_humedad['index']*tabla_frec_humedad['humedad (%)']).sum() #sum_x=4816.27

In [ ]:
tabla_frec_acc_humedad=tabla_frec_humedad.copy()
tabla_frec_acc_humedad['frec_acc']=0 #frec_acc ES EL MISMO NOMBRE PARA TODAS PARA GENERALIZAR LOS DATAFRAMES
tabla_frec_acc_humedad_reset=tabla_frec_acc_humedad.reset_index()

frec_acc1=[]
frec_acc1.append(tabla_frec_acc_humedad_reset['humedad (%)'][0])

for i in range(1,len(tabla_frec_acc_humedad_reset)):
  frec_acc1.append(tabla_frec_acc_humedad_reset['humedad (%)'][i]+frec_acc1[i-1])

tabla_frec_acc_humedad_reset['frec_acc']=frec_acc1.copy() #frec_acc ES EL MISMO NOMBRE PARA TODAS PARA GENERALIZAR LOS DATAFRAMES
tabla_frec_acc_humedad_reset

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x=df.columns[1], data = df)
plt.title('diagrama de caja de la humedad (%)')

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(x=df.columns[1], data = df)
plt.title('diagrama de violin de la humedad (%)')

In [ ]:
plt.figure(figsize=(50, 8))
sns.countplot(x=df.columns[2], data = df)
plt.title('temperatura (C°)')

In [ ]:
tabla_frec_temperatura=df.sort_values('temperatura (C°)')['temperatura (C°)'].value_counts().reset_index().sort_values('index')
tabla_frec_temperatura

In [ ]:
tabla_frec_temperatura.sum()# n=63 ; sum_x must be corrected as in the next line of code

In [ ]:
(tabla_frec_temperatura['index']*tabla_frec_temperatura['temperatura (C°)']).sum() #sum_x=1098.7

In [ ]:
tabla_frec_acc_temperatura=tabla_frec_temperatura.copy()
tabla_frec_acc_temperatura['frec_acc']=0 #frec_acc ES EL MISMO NOMBRE PARA TODAS PARA GENERALIZAR LOS DATAFRAMES
tabla_frec_acc_temperatura_reset=tabla_frec_acc_temperatura.reset_index()

frec_acc2=[]
frec_acc2.append(tabla_frec_acc_temperatura_reset['temperatura (C°)'][0])

for i in range(1,len(tabla_frec_acc_temperatura_reset)):
  frec_acc2.append(tabla_frec_acc_temperatura_reset['temperatura (C°)'][i]+frec_acc2[i-1])

tabla_frec_acc_temperatura_reset['frec_acc']=frec_acc2.copy() #frec_acc ES EL MISMO NOMBRE PARA TODAS PARA GENERALIZAR LOS DATAFRAMES
tabla_frec_acc_temperatura_reset

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x=df.columns[2], data = df)
plt.title('diagrama de caja de la temperatura (C°)')

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(x=df.columns[2], data = df)
plt.title('diagrama de violin de la temperatura (C°)')

In [ ]:
plt.figure(figsize=(50, 8))
sns.countplot(x=df.columns[3], data = df)
plt.title('amoniaco (ppm)')

In [ ]:
tabla_frec_amoniaco=df.sort_values('amoniaco (ppm)')['amoniaco (ppm)'].value_counts().reset_index().sort_values('index') # tabla de frecuencia
tabla_frec_amoniaco

In [ ]:
tabla_frec_amoniaco.sum() # n=63 ; sum_x must be corrected as in the next line of code

In [ ]:
(tabla_frec_amoniaco['index']*tabla_frec_amoniaco['amoniaco (ppm)']).sum() #sum_x=605.9

In [ ]:
tabla_frec_acc_amoniaco=tabla_frec_amoniaco.copy()
tabla_frec_acc_amoniaco['frec_acc']=0 #frec_acc ES EL MISMO NOMBRE PARA TODAS PARA GENERALIZAR LOS DATAFRAMES
tabla_frec_acc_amoniaco_reset=tabla_frec_acc_amoniaco.reset_index()

frec_acc3=[]
frec_acc3.append(tabla_frec_acc_amoniaco_reset['amoniaco (ppm)'][0])

for i in range(1,len(tabla_frec_acc_amoniaco_reset)):
  frec_acc3.append(tabla_frec_acc_amoniaco_reset['amoniaco (ppm)'][i]+frec_acc3[i-1])

tabla_frec_acc_amoniaco_reset['frec_acc']=frec_acc3.copy() #frec_acc ES EL MISMO NOMBRE PARA TODAS PARA GENERALIZAR LOS DATAFRAMES
tabla_frec_acc_amoniaco_reset

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x=df.columns[3], data = df)
plt.title('diagrama de caja del amoniaco (ppm)')

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(x=df.columns[3], data = df)
plt.title('diagrama de violin del amoniaco (ppm)')

In [ ]:
plt.figure(figsize=(50, 8))
sns.countplot(x=df.columns[4], data = df)
plt.title('viento (m/s )')

In [ ]:
tabla_frec_viento=df.sort_values('viento (m/s )')['viento (m/s )'].value_counts().reset_index().sort_values('index') # tabla de frecuencia
tabla_frec_viento

In [ ]:
tabla_frec_viento.sum() # n=63 ; sum_x must be corrected as in the next line of code

In [ ]:
(tabla_frec_viento['index']*tabla_frec_viento['viento (m/s )']).sum() #sum_x=11.7

In [ ]:
tabla_frec_acc_viento=tabla_frec_viento.copy()
tabla_frec_acc_viento['frec_acc']=0 #frec_acc ES EL MISMO NOMBRE PARA TODAS PARA GENERALIZAR LOS DATAFRAMES
tabla_frec_acc_viento_reset=tabla_frec_acc_viento.reset_index()

frec_acc4=[]
frec_acc4.append(tabla_frec_acc_viento_reset['viento (m/s )'][0])

for i in range(1,len(tabla_frec_acc_viento_reset)):
  frec_acc4.append(tabla_frec_acc_viento_reset['viento (m/s )'][i]+frec_acc4[i-1])

tabla_frec_acc_viento_reset['frec_acc']=frec_acc4.copy() #frec_acc ES EL MISMO NOMBRE PARA TODAS PARA GENERALIZAR LOS DATAFRAMES
tabla_frec_acc_viento_reset


In [ ]:
#PERCENTILES(%): esta función para el caso del viento(cuarta variable) (cuyos resultados si se ecuentran en los datos) hace algo similar a lo que hace la función definida en python como quantile() que arroja resultados exactos (cuyos valores no necesariamente se encuentran en los datos)

def percentil4(porcentaje):#porcentaje en valor decimal (si es 50%, se pasa solo 50/100=0.5), igual que la función quantile()
  div=n/100
  for fila in range(len(tabla_frec_acc_viento_reset)):
    if (tabla_frec_acc_viento_reset['frec_acc'][fila]>=porcentaje*100*div):
      return tabla_frec_acc_viento_reset['index'][fila]

In [ ]:
print(percentil4(0.25),percentil4(0.50),percentil4(0.75)) # solo para el caso del viento

In [ ]:
#GENERALIZANDO LA FUNCIÓN ANTERIOR PARA LAS CUATRO TABLAS de FREC ACC YA CONSTRUIDAS:

def percentil(tabla_frec_acc,porcentaje):#porcentaje en valor decimal (si es 50%, se pasa solo 50/100=0.5), igual que la función quantile()
  div=n/100
  for fila in range(len(tabla_frec_acc)):
    if (tabla_frec_acc['frec_acc'][fila]>=porcentaje*100*div):
      return tabla_frec_acc['index'][fila]

In [ ]:
tablas_frec_acc=[tabla_frec_acc_humedad_reset, tabla_frec_acc_temperatura_reset,tabla_frec_acc_amoniaco_reset,tabla_frec_acc_viento_reset] #lista de dataframes

In [ ]:
percentil(tabla_frec_acc_viento_reset,0.75)

In [ ]:
percentil4(0.75)

In [ ]:
percentil4(0.88) #cualquier otro porcentaje

In [ ]:
df.quantile(0.88)

In [ ]:
[percentil(elem,0.75) for elem in tablas_frec_acc]

In [ ]:
[[percentil(elem1,elem2) for elem1 in tablas_frec_acc] for elem2 in [0.25,0.50,0.75]] # lista de listas (aquí se arrojan tres listas, 1 por cada percentil, y dentro ce cada lista las cuatro variables)

In [ ]:
[[percentil(elem1,elem2) for elem2 in [.25,.5,.75]] for elem1 in tablas_frec_acc] #.5=0.5=0.50, lista de listas (aquí se arrojan cuatro listas, 1 por cada variable, y dentro ce cada lista los tres percentiles)

In [ ]:
#AQUI COMPROBAMOS QUE LO ANTERIOR ESTÁ CORRECTO
df.quantile([0.25, 0.50, 0.75], axis=0)

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x=df.columns[4], data = df)
plt.title('diagrama de caja del viento (m/s )')

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(x=df.columns[4], data = df)
plt.title('diagrama de violin del viento (m/s )')

# Ahora la relación entre amoniaco(y) y viento(x): y=f(x)

In [ ]:
plt.figure(figsize=(25, 8))
ax = sns.scatterplot(
    x=df.columns[4],
    y=df.columns[3],
    data=df,
)

In [ ]:
plt.figure(figsize=(25, 8))
ax = sns.boxplot(
    x=df.columns[4],
    y=df.columns[3],
    data=df,
)

In [ ]:
plt.figure(figsize=(25, 8))
ax = sns.violinplot(
    x=df.columns[4],
    y=df.columns[3],
    data=df,
)

In [ ]:
plt.figure(figsize=(25, 8))
ax = sns.lineplot(
    x=df.columns[4],
    y=df.columns[3],
    data=df,
)

In [ ]:
plt.scatter(df['viento (m/s )'],df['amoniaco (ppm)'])
plt.title("amoniaco (ppm) vs viento (m/s )", fontsize=20, verticalalignment='bottom');
plt.xlabel("viento (m/s )");
plt.ylabel("amoniaco (ppm)");

In [ ]:
sns.lmplot(x = 'viento (m/s )', y = 'amoniaco (ppm)', data = df, scatter_kws = {'color': (174/255,199/255,14/255)})
plt.title("amoniaco (ppm) vs. viento (m/s )", fontsize=20, verticalalignment='bottom')
plt.xlabel("viento (m/s )")
plt.ylabel("amoniaco (ppm)")

# Regresión Lineal Simple: y=mx+b +error (m=pendiente, b=intercepto)

In [ ]:
### Cargando los paquetes relevantes
import pandas                  as pd
from   scipy import stats
import numpy                   as np
import matplotlib.pyplot       as plt
import seaborn                 as sns
import statsmodels.formula.api as sm
#import chart_studio.plotly     as py
# https://community.plot.ly/t/solved-update-to-plotly-4-0-0-broke-application/26526/2
import os

%matplotlib inline
plt.style.use('ggplot')

In [ ]:
df.columns

# Renombrando las columnas

In [ ]:
df2=df.rename(columns={'Tiempo de Medición  (m/d/A - hora)':'tiempo_medicion_timestamp', 'humedad (%)':'humedad_percent', 'temperatura (C°)':'temperatura_celsius','amoniaco (ppm)':'amoniaco_ppm', 'viento (m/s )':'viento_mt_per_sec'})
df2.columns

In [ ]:
df2

In [ ]:
sum_x=df2['viento_mt_per_sec'].sum() #sum_x
sum_x

In [ ]:
sum_y=df2['amoniaco_ppm'].sum() #sum_y
sum_y

In [ ]:
sum_xy=(df2['viento_mt_per_sec']*df2['amoniaco_ppm']).sum() #sum_xy
sum_xy

In [ ]:
sum([df2['viento_mt_per_sec'][i]*df2['amoniaco_ppm'][i] for i in range(63)]) # usando una lista comprimida da el mismo resultado de sum_xy

In [ ]:
sum_x2=(df2['viento_mt_per_sec']**2).sum() #sum_x^2
sum_x2

In [ ]:
#ya conociamos n=len(df)=63
def calculo_regresion_lineal_simple(n,sum_x,sum_y,sum_xy,sum_x2): # y_estimada = pendiente*x + intercepto = m*x + b
  m=(n*sum_xy-sum_x*sum_y)/(n*sum_x2-(sum_x)**2)
  b= (sum_y-m*sum_x)/n
  return m, b

In [ ]:
(pendiente, intercepto) = calculo_regresion_lineal_simple(n,sum_x,sum_y,sum_xy,sum_x2)
print('pendiente= ',pendiente) # debe ser igual al valor de viento_mt_per_sec como se muestra justo en el siguiente resultado de la celda
print('intercepto= ',intercepto) # debe ser igual al valor de Intercept como se muestra justo en el siguiente resultado de la celda
print(f'La ecuación lineal estimada es: y_estimada = m * x + b = {pendiente} * x + {intercepto}')

In [ ]:
def calculo_y_estimada(x):
  (m,b)=calculo_regresion_lineal_simple(n,sum_x,sum_y,sum_xy,sum_x2)
  y_estimada=m*x+b
  return y_estimada

In [ ]:
y_estimada=calculo_y_estimada(0.2)
y_estimada

In [ ]:
#Definición del modelo
modelo1 = 'amoniaco_ppm~viento_mt_per_sec'

#Entrenamiento del modelo con la librería 'statsmodels'
lm1   = sm.ols(formula = modelo1, data = df2).fit()
print(lm1.summary())

De acuerdo a lo anterior, obtenemos m y b:

In [ ]:
m=-0.9504 # pendiente asociada a la var independiente (x=viento)
b=9.7940  # intercepto (punto de corte con el eje y(amoniaco) cuando x(viento)=0)

In [ ]:
sns.lmplot(x = 'viento_mt_per_sec', y = 'amoniaco_ppm', data = df2, scatter_kws = {'color': (174/255,199/255,14/255)})
plt.title("amoniaco_ppm vs. viento_mt_per_sec", fontsize=20, verticalalignment='bottom')
plt.axhline(df2['amoniaco_ppm'].mean(), ls='--',color = 'b')
plt.xlabel("viento_mt_per_sec")
plt.ylabel("amoniaco_ppm")

In [ ]:
df2['viento_mt_per_sec'].head(3)

In [ ]:
#comparemos las predicciones del modelo de python con las de mi modelo (ver siguiente celda):
calculo_y_estimada(df2['viento_mt_per_sec'])

In [ ]:
#comparemos las predicciones del modelo de python con las de mi modelo (ver siguiente celda):
y_estimada=[calculo_y_estimada(xi) for xi in df2['viento_mt_per_sec']]
y_estimada

In [ ]:
y_estimada=lm1.predict()
y_estimada

# Grafica de la recta de regresión:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(0,1,0.01)
y = m*x + b

plt.plot(x,y)
plt.xlabel('x')
plt.ylabel('y')
plt.title('y=mx+b')
plt.show()

# Error estándar (Se) de y_estimada (Ye), Coeficiente de Determinación (r^2) y Coeficiente de Correlación Lineal (r=raiz_cuadrada(r^2)):

- Si r^2 --> 1, modelo "perfecto" (confiable)
- Si r^2 --> 0.5, modelo "regular" (confiable al 50%)
- Si r^2 --> 0, modelo "imperfecto" (no confiable)

- Si r --> 1, hay correlación positiva. x,y tienen una relacion directa: m es positiva
-Si r--> 0, la correlación entre x, y tiende a 0 (se aproxima a que no hay relación, y sería una linea horizontal de pendiente aproximadamente cero)
- Si r --> -1, hay correlación negativa. x,y tienen una relacion inversa: m es negativa

In [ ]:
df2.head(3)

In [ ]:
#dataframe auxiliar solo para las vars y=amoniaco_ppm, x=viento_mt_per_sec
df2_aux=df2[['amoniaco_ppm','viento_mt_per_sec']].copy()
df2_aux.head(3)

In [ ]:
df2_aux_final=df2_aux.rename(columns={'amoniaco_ppm':'y', 'viento_mt_per_sec':'x'})
df2_aux_final.head(3)

https://www.analyticslane.com/2019/03/25/como-eliminar-columnas-y-filas-en-un-dataframe-pandas/#:~:text=Para%20eliminar%20una%20columna%20de,de%20esta%20propiedad%20es%201.

In [ ]:
#ELIMINAR FILAS: df.drop([[1, 2]], axis=0)
#ELIMINAR COLUMNA: df.drop(['name_of_col'], axis=1)
df2_aux_final2=df2_aux_final.drop(['y'],axis=1)
df2_aux_final2.head()

In [ ]:
df2_aux_final2['y']=df2_aux_final['y']
df2_aux_final2['y_estimada']=y_estimada
df2_aux_final2['y-y_estimada']=df2_aux_final2['y']-df2_aux_final2['y_estimada']
df2_aux_final2['(y-y_estimada)^2']=df2_aux_final2['y-y_estimada']**2 #aquí se eleva a la 2d
df2_aux_final2.head()

In [ ]:
(df2_aux_final2['(y-y_estimada)^2']).sum() #ya está elevada al cuadrado

In [ ]:
(2389.765821494572/(63-2))**0.5

In [ ]:
def calculo_error_estandar_y_estimada():#y,y_estimada son los nombres de las variables del df2_aux_final2 (OJO)
  sum_y_menos_ye_cuadrado=(df2_aux_final2['(y-y_estimada)^2']).sum() #ya está elevada al cuadrado
  Se=(sum_y_menos_ye_cuadrado/(n-2))**0.5 #n=len(y)=len(y_estimada)=len(df2_aux_final2)
  return Se

Se=calculo_error_estandar_y_estimada()
print('Se= ',Se)

In [ ]:
Se**2

In [ ]:
(df2_aux_final2['y'].std())**2

In [ ]:
1-(39.17648887696019/38.624690220174095) # Tomado del modelo de python: Adj. R-squared:                 -0.014

In [ ]:
def calculo_coef_determinacion(Se):
  r2=1-(Se**2/(df2_aux_final2['y'].std())**2)
  return r2

r2=calculo_coef_determinacion(Se)
print('r^2= ',r2)

In [ ]:
def calculo_coef_corelacion_lineal(r2):
  r=-r2**0.5  #ajustado para valor negativo porque la pendiente dió negativa
  return r

r=calculo_coef_corelacion_lineal(abs(r2)) #ajustado para valor positivo
print('r= ',r)

In [ ]:
datos=df2.copy()

In [ ]:
datos.head(3)

# Covarianza:

*Tomado de: https://www.odiolaestadistica.com/estadistica-python/covarianza/#:~:text=En%20python%2C%20utilizamos%20el%20m%C3%A9todo,las%20variables%20que%20nos%20interesan:*

En python, utilizamos el método cov para calcular la matriz de varianzas y covarianzas de las variables numéricas de un DataFrame. Si luego queremos una covarianza en particular, podemos utilizar .loc con los nombres de las variables que nos interesan.


In [ ]:
cov_mat = datos.cov()
cov_mat

In [ ]:
cov_mat.loc['amoniaco_ppm',	'viento_mt_per_sec']

**La covarianza de una variable consigo misma es la varianza de la variable:**

In [ ]:
cov_mat.loc['viento_mt_per_sec',	'viento_mt_per_sec'] # cov(x,x)=var(x)=Sx^2=std(x)^2

In [ ]:
datos['viento_mt_per_sec'].var()

In [ ]:
(datos['viento_mt_per_sec'].std())**2

In [ ]:
cov_mat.loc['amoniaco_ppm',	'amoniaco_ppm']#cov(y,y)=var(y)=Sy^2=std(y)^2

In [ ]:
df2_aux_final2['y_estimada'].var()

In [ ]:
(df2_aux_final2['y_estimada'].std())**2

In [ ]:
df2_aux_final2['y-y_estimada'].var() #residuo=(y-y_estimada) #y=y_observada

In [ ]:
df2_aux_final2.head(3)

In [ ]:
cov_mat2 = df2_aux_final2[['x','y','y_estimada', 'y-y_estimada']].cov()
cov_mat2

In [ ]:
cov_mat2.loc['y_estimada', 'y_estimada']#cov(ye,ye)=var(ye)=Se^2=std(ye)^2

In [ ]:
cov_mat2.loc['y-y_estimada', 'y-y_estimada']#cov(residuo,residuo)=var(y-ye)=Se^2=std(y-ye)^2 #Se=Error(desviación) estandar

In [ ]:
(df2_aux_final2['y-y_estimada'].std())**2

In [ ]:
cov_mat2.loc['y', 'y']#cov(y,y)=var(y)=Sy^2=std(y)^2

In [ ]:
var_residual=df2_aux_final2['y-y_estimada'].var()
var_residual

In [ ]:
var_y=df2_aux_final2['y'].var()
var_y

**Coeficiente de determinación y correlación a partir de los resultados anteriores:**

In [ ]:
r2=1-(var_residual/var_y)
r2

**Como r=(+/-)(raiz_cuadrada(r^2)), entonces se debe considerar los dos caasos (positivo y negativoo), y finalmente escoger el signo dependiendo de la pendiente antes calculada, que para este caso dio negativa:**

In [ ]:
r_pos=r2**0.5
r_pos

In [ ]:
r_neg=-(r2**0.5)
r_neg # se comprueba que da muy aprozimado al calculado en la celda que sigue

**Coeficiente de correlación de Person apartir de la covarianza(x,y):**

ρ=cov(x,y)/σx*σy;

σ(u)=std(u)



In [ ]:
ρ=cov_mat2.loc['x',	'y']/(df2_aux_final2['x'].std()*df2_aux_final2['y'].std())
ρ

**Otras alternativas para calcular la covarianza:**

1. Sxy = cov(x,y) = (1/(n-1))( ∑(xi)(yi) ) - (n/(n-1))(x_prom)(y_prom)

2. Covarianza muestral = cov(x,y) = (1/(n-1)) ∑(xi−x_prom)(yi−y_prom)



In [ ]:
#alternativa 1:

def cov_alt1(x, y): #alt1=alternativa1
  n = len(x)
  return ((x * y).sum() / (n - 1)) - ((n / (n - 1)) * x.mean() * y.mean())

In [ ]:
cov_alt1(df2_aux_final2['x'],df2_aux_final2['y'])

In [ ]:
#alternativa 2:
def cov_alt2(x, y): #alt1=alternativa1
  n = len(x)
  return ((1 / (n - 1)) * ( (x-x.mean()) * (y-y.mean())).sum() )

In [ ]:
cov_alt2(df2_aux_final2['x'],df2_aux_final2['y'])

In [ ]:
cov_mat2.loc['x','y'] # da el mismo resultado que usando las dos fórmulas estadísticas definidas por mi en las anteriores celdas

**Aunque ya teniendo las las dos formulas anteriores no sea necesario hacer lo siguiente, retomemos de df_final xi-x_prom4 (=x-x_prom) y xi-x_prom3 (=y-y_prom) para visualizar mejor y calcular lo mismo que hace cov_alt2(x,y):**

In [ ]:
df_final.head(3)

In [ ]:
cov_alt2_tabla=((df_final['xi-xprom4']*df_final['xi-xprom3']).sum())/(n-1)
cov_alt2_tabla

# Matriz de Correlaciones:

In [ ]:
# Matriz de correlación
# ==============================================================================
corr_matrix = datos.corr(method='pearson')
corr_matrix

Las matrices de correlación tienen el inconveniente de tener un tamaño notable cuando se dispone de muchas variables. Para facilitar la identificación de pares de variables con correlaciones altas, es conveniente convertirlas en formato de tabla larga (tidy).

In [ ]:
def tidy_corr_matrix(corr_mat):
    '''
    Función para convertir una matriz de correlación de pandas en formato tidy.
    '''
    corr_mat = corr_mat.stack().reset_index()
    corr_mat.columns = ['variable_1','variable_2','r']
    corr_mat = corr_mat.loc[corr_mat['variable_1'] != corr_mat['variable_2'], :]
    corr_mat['abs_r'] = np.abs(corr_mat['r'])
    corr_mat = corr_mat.sort_values('abs_r', ascending=False)

    return(corr_mat)

tidy_corr_matrix(corr_matrix)

In [ ]:
# Heatmap matriz de correlaciones
# ==============================================================================
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))

sns.heatmap(
    corr_matrix,
    annot     = True,
    cbar      = False,
    annot_kws = {"size": 8},
    vmin      = -1,
    vmax      = 1,
    center    = 0,
    cmap      = sns.diverging_palette(20, 220, n=200),
    square    = True,
    ax        = ax
)

ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation = 45,
    horizontalalignment = 'right',
)

ax.tick_params(labelsize = 10)

# Correlación:

In [ ]:
datos.head(3)

**Coeficientes de correlación lineal:**

*Tomado de: https://www.cienciadedatos.net/documentos/pystats05-correlacion-lineal-python.html:*

"Los coeficientes de correlación lineal son estadísticos que cuantifican la asociación lineal entre dos variables numéricas. Existen diferentes tipos, de entre los que destacan el Pearson, Rho de Spearman y Tau de Kendall. Todos ellos comparten que:

1. Su valor está comprendido en el rango [+1 , -1]. Siendo +1 una correlación positiva perfecta y -1 una correlación negativa perfecta.

2. Se emplean como medida de la fuerza de asociación entre dos variables (tamaño del efecto):

- 0: asociación nula
- 0.1: asociación pequeña
- 0.3: asociación mediana
- 0.5: asociación moderada
- 0.7: asociación alta
- 0.9: asociación muy alta

3. Desde el punto de vista práctico, las principales diferencias entre estos tres coeficientes son:

- La correlación de Pearson funciona bien con variables cuantitativas que tienen una distribución normal o próxima a la normal. Es más sensible a los valores extremos que las otras dos alternativas.

- La correlación de Spearman se emplea con variables cuantitativas (continuas o discretas). En lugar de utilizar directamente el valor de cada variable, los datos son ordenados y reemplazados por su respectivo orden [ranking](https://es.wikipedia.org/wiki/Ranking). Es un método no paramétrico muy utilizado cuando no se satisface la condición de normalidad necesaria para aplicar la correlación de Pearson.

- La correlación de Kendall es otra alternativa no paramétrica que, al igual que la correlación de Spearman, utiliza la ordenación de las observaciones [ranking](https://). Es recomendable cuando se dispone de pocos datos y muchos de ellos ocupan la misma posición en el rango, es decir, cuando hay muchas ligaduras."




In [ ]:
import pingouin as pg

In [ ]:
# Cálculo de correlación lineal
# ==============================================================================
pg.corr(x=datos['viento_mt_per_sec'], y=datos['amoniaco_ppm'], method='pearson')

In [ ]:
# Cálculo de correlación lineal
# ==============================================================================
pg.corr(x=datos['viento_mt_per_sec'], y=datos['amoniaco_ppm'], method='spearman')

In [ ]:
# Cálculo de correlación lineal
# ==============================================================================
pg.corr(x=datos['viento_mt_per_sec'], y=datos['amoniaco_ppm'], method='kendall') #POR LOS DATOS MANEJADOS EN ESTE CASO, ESTE METODO ES EL QUE MAS SE AJUSTA Y ARROJA RESULTADOS SIMILARES A LOS DEL MODELO:

#Influencia de las covariables humedad y temperatura:

In [ ]:
# Cálculo de correlación lineal parcial
# ==============================================================================
pg.partial_corr(data=datos, x='viento_mt_per_sec', y='amoniaco_ppm', covar='humedad_percent', method='kendall')

In [ ]:
# Cálculo de correlación lineal parcial
# ==============================================================================
pg.partial_corr(data=datos, x='viento_mt_per_sec', y='amoniaco_ppm', covar='temperatura_celsius', method='kendall')

#Conclusiones:

-La correlación lineal es un método estadístico que permite cuantificar la relación lineal existente entre dos variables. Existen varios estadísticos, llamados coeficientes de correlación lineal, desarrollados con el objetivo de medir este tipo de asociación, algunos de los más empleados son Pearson, Spearman y Kendall.

Con frecuencia, los estudios de correlación lineal preceden a análisis más complejos, como la creación de [modelos de regresión](https://www.cienciadedatos.net/documentos/py10-regresion-lineal-python.html). Primero, se analiza si las variables están correlacionadas y, en caso de estarlo, se procede a generar modelos.

Es importante destacar que, la existencia de correlación entre dos variables, no implica necesariamente causalidad. La asociación observada puede deberse a un tercer factor ([confounder](https://en.wikipedia.org/wiki/Confounding)).

- r^2 se mide en tantos por ciento. Si la varianza residual (Se^2) es cero, el modelo explica el 100% del valor de la variable; si coincide con la varianza de la variable dependiente (Sy^2), el modelo no explica nada y el coeficiente de determinación es del 0%

- Si r < 0, hay correlación negativa : las dos variables se correlacionan en sentido inverso.A valores altos de una de ellas le suelen corresponder valor bajos de la otra y viceversa.Cuánto más próximo a -1 esté el coeficiente de correlación más patente será esta covariación extrema.Si r= -1 hablaremos de correlación negativa perfecta lo que supone una determinación absoluta entre las dos variables ( en sentido inverso): Existe una relación funcional perfecta entre ambas(una relación lineal de pendiente negativa).

- Si r > 0, hay correlación positiva: las dos variables se correlacionan en sentido directo.A valores altos de una le corresponden valores altos de la otra e igualmente con los valores bajos.Cuánto más próximo a +1 esté el coeficiente de correlación más patente será esta covariación.Si r = 1 hablaremos de correlación positiva perfecta lo que supone una determinación absoluta entre las dos variables (en sentido directo):Existe una relación lineal perfecta ( con pendiente positiva).

- Si r = 0, se dice que las variables están incorrelacionadas: no puede establecerse ningún sentido de covariación.

- La correlación entre el viento(x) y la concetración del amoniaco(y) es muy baja (r=-0.14975) y NO tan significativa (pvalue≈12% > 5%). Sin embargo, cuando se estudia su relación bloqueando otras variables como humedad y temperatura, a pesar de que la relación ahora ES significativa (pvalue≈3.5% < 5%, pvalue≈3.3% < 5%), los coeficientes de correlación aumentan en valor absoluto y conservan el sentido negativo (r=-0.181772,r=-0.184577), respectivalente. Entronces, se puede afirmar que la casi nula relación lineal existente entre el viento(x) y la concentración de amoniaco(y) está influenciada por el efecto de otras covariables como la humedad y temperatura.

**Referencias:**

- https://es.wikipedia.org/wiki/Coeficiente_de_determinaci%C3%B3n
- https://www.uv.es/ceaces/base/descriptiva/coefcorre.htm
- https://www.cienciadedatos.net/documentos/24_correlacion_y_regresion_lineal
- https://www.cienciadedatos.net/documentos/pystats05-correlacion-lineal-python.html
- https://www.odiolaestadistica.com/estadistica-python/covarianza/#:~:text=En%20python%2C%20utilizamos%20el%20m%C3%A9todo,las%20variables%20que%20nos%20interesan


# Información de sesión:

In [ ]:
!pip install sinfo
from sinfo import sinfo
sinfo()

**¡Terminado!**


